In [1]:
import cv2
import imutils
import numpy as np

In [2]:
#global variables
bg = None

In [3]:
def segment(image, threshold=25):
    global bg
    
    diff = cv2.absdiff(bg.astype("uint8"), image)

    thresholded = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)[1]

    ( cnts, _) = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(cnts) == 0:
        return
    else:
        segmented = max(cnts, key=cv2.contourArea)
        return (thresholded, segmented)

In [4]:
def run_avg(image, aWeight):
    global bg
    
    if bg is None:
        bg = image.copy().astype("float")
        return

    cv2.accumulateWeighted(image, bg, aWeight)

In [6]:
aWeight = 0.5

camera = cv2.VideoCapture(0)

top, right, bottom, left = 10, 350, 225, 590

num_frames = 0

while(True):
    (grabbed, frame) = camera.read()

    frame = imutils.resize(frame, width=700)

    frame = cv2.flip(frame, 1)

    clone = frame.copy()

    (height, width) = frame.shape[:2]

    roi = frame[top:bottom, right:left]
    

    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (7, 7), 0)

    if num_frames < 30:
        run_avg(gray, aWeight)
    else:
        hand = segment(gray)

        if hand is not None:
            (thresholded, segmented) = hand

            cv2.drawContours(clone, [segmented + (right, top)], -1, (0, 0, 255))
            cv2.imshow("Thesholded", thresholded)
            cv2.putText(clone, str('Team-1'), (70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)


    cv2.rectangle(clone, (left, top), (right, bottom), (0,255,0), 2)

    num_frames += 1

    cv2.imshow("Video Feed", clone)

    keypress = cv2.waitKey(1) & 0xFF

    if keypress == ord("q"):
        outpath = "Hanif_Save.jpg"
        cv2.imwrite(outpath, thresholded)
        break

camera.release()
cv2.destroyAllWindows()